In [1]:
import numpy as np
import nibabel as nib
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from scipy.io import loadmat
from sklearn.model_selection import train_test_split, cross_val_score, KFold
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC

## Import and load fMRI data

In [2]:
#import and load fmri data
im_path = "sub-01/ses-test/func/sub-01_ses-test_task-fingerfootlips_bold.nii"
img = nib.load(im_path)
fmri_data = img.get_fdata()

In [4]:
#load labels
label_path = 'label.mat'
mat_labels = loadmat(label_path)

## Threshold and mask the data

In [3]:
#set threshold and threshold data
threshold_value = 255
thresholded_data = np.where(fmri_data >= threshold_value, 1, 0)
#apply mask to the data
brain_mask = np.sum(thresholded_data, axis=-1) > 0
reshaped_brain_mask = np.expand_dims(brain_mask, axis=-1)
masked_data = fmri_data * reshaped_brain_mask

## Preprocess labels and training data

In [5]:
#preprocess labels into a list for training
arr_labels = mat_labels['label']
labels = []
for i in range(len(arr_labels)):
    labels.append(arr_labels[i][0])

In [6]:
#reshape the data to (features, samples) for training
reshaped_data = np.reshape(masked_data, (64 * 64 * 30, 184)).T

#convert labels to a NumPy array
labels_array = np.array(labels)

## Conduct PCA

In [7]:
#set pca parameter and initialize
n_components = 45
#initialize pca and fit to training data
pca = PCA(n_components = n_components, svd_solver ='randomized',
          whiten = True).fit(reshaped_data)

#apply pca to data
X_pca = pca.transform(reshaped_data)

## Initialize SVM model

In [8]:
#initialize svm model
svm = SVC(kernel='linear')
#create a pipeline with PCA and SVM
pipeline = Pipeline([('pca', pca), ('svm', svm)])

#use 10-fold cross-validation
kf = KFold(n_splits=10, shuffle=True, random_state=42)

## Cross Validation

In [9]:
#perform cross-validation
cv_scores = cross_val_score(pipeline, X_pca, labels_array, cv=kf)

#display the cross-validation scores
print("Cross-Validation Scores:", cv_scores)
print("\nMean Accuracy:", np.mean(cv_scores))

Cross-Validation Scores: [0.68421053 0.94736842 0.84210526 1.         0.94444444 0.94444444
 0.83333333 0.94444444 0.83333333 0.88888889]

Mean Accuracy: 0.8862573099415204
